In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
import numpy as np
import pandas as pd
import gc

/home/ramezani/anaconda/lib/python2.7/site-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [3]:
path = "../../../../data/"
print("Read the properties and merge with coord")
coord = pd.read_csv(path + 'renamed_properties_2017.csv', usecols = ["id_parcel", "latitude", "longitude"])
print "Size of the properties data frame: ", coord.shape

Read the properties and merge with coord
Size of the properties data frame:  (2985217, 3)


In [4]:
features = ["id_parcel", "area_lot", "tax_property", "build_year", "area_total_calc", "tax_building", "tax_total"]

prop = pd.read_csv(path + 'renamed_properties_2017.csv', usecols = features)
print "Size of the properties data frame: ", prop.shape

Size of the properties data frame:  (2985217, 7)


In [5]:
index = pd.isnull(coord["latitude"])
coord = coord[~index]
prop = prop[~index]
print "Size of the properties data frame: ", coord.shape
print "Size of the properties data frame: ", prop.shape

Size of the properties data frame:  (2982285, 3)
Size of the properties data frame:  (2982285, 7)


In [6]:
coord.latitude = coord.latitude / 1e6
coord.longitude = coord.longitude / 1e6

In [7]:
# Read Train
print("Read train file and merge with properties to generate train file and taraget")
path = '../../../../data/'
train_2016 = pd.read_csv(path + 'renamed_train_2016.csv', parse_dates=['date'])
train_2016 = train_2016.drop("Unnamed: 0", axis=1)
train_coord_2016 = pd.merge(train_2016, coord, how='left', on='id_parcel')
train_coord_2016 = train_coord_2016.drop(['date', 'logerror'], axis=1)

train_2017 = pd.read_csv(path + 'renamed_train_2017.csv', parse_dates=['date'])
train_2017 = train_2017.drop("Unnamed: 0", axis=1)
train_coord_2017 = pd.merge(train_2017, coord, how='left', on='id_parcel')
train_coord_2017 = train_coord_2017.drop(['date', 'logerror'], axis=1)

Read train file and merge with properties to generate train file and taraget


In [9]:
# Concatenate 2016 and 2017 train
train_coord = pd.concat([train_coord_2016, train_coord_2017])

In [12]:
index = pd.isnull(train_coord["latitude"])
train_coord = train_coord[~index]

In [14]:
from sklearn.neighbors import NearestNeighbors
from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()
print "number of cores " + str(num_cores)

number of cores 6


In [15]:
def splitDataFrameIntoSmaller(df, chunkSize = 100000): 
    listOfDf = list()
    numberChunks = len(df) // chunkSize + 1
    for i in range(numberChunks):
        listOfDf.append(i*chunkSize)
    listOfDf.append(len(df))
    return listOfDf

split_index = splitDataFrameIntoSmaller(prop, chunkSize=coord.shape[0]/num_cores)
print split_index

[0, 333, 666, 999, 1332, 1665, 1998, 2000]


In [16]:
def knn_avg(n1, n2, cc):
    avg_list = []
    for ii in range(n1, n2):
        index = np.setdiff1d(indices[ii], ii)
        avg = prop.loc[index, cc].mean()
        avg_list.append(avg)
    return avg_list

In [17]:
features = ["area_lot", "tax_property", "build_year", "area_total_calc", "tax_building", "tax_total"]
feat_avg = ["area_lot_avg", "tax_property_avg", "build_year_avg", "area_total_calc_avg", 
            "tax_building_avg", "tax_total_avg"]

In [18]:
knn = NearestNeighbors(n_neighbors=10000, algorithm='ball_tree', 
                        n_jobs = num_cores).fit(coord.loc[:, ["latitude", "longitude"]])
knn_train = NearestNeighbors(n_neighbors=10000, algorithm='ball_tree', 
                        n_jobs = num_cores).fit(train_coord.loc[:, ["latitude", "longitude"]])

In [19]:
for k in [5]:
#for k in [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]:
    ss = "k" + str(k)
    print "Folder " + ss
    
    #get the distances, indices, and compute the min and max ditances
    distances, indices  = knn.kneighbors(coord.loc[:, ["latitude", "longitude"]], n_neighbors = k,
                                          return_distance = True)
    
    rmax = []
    rmin = []
    minmax_ratio = []
    for ii in distances:
        rmax.append(max(ii))
        rmin.append(min(ii[1:]))
        minmax_ratio.append(max(ii)/min(ii[1:]))
        
    prop['rmax'] = rmax
    prop['rmin'] = rmin
    prop['minmax_ratio'] = minmax_ratio
    
    dist_train, ind_train  = knn_train.kneighbors(coord.loc[:, ["latitude", "longitude"]], n_neighbors = k,
                                          return_distance = True)
    
    rtran = []
    for ii in dist_train:
        rtran.append(max(ii))
        
    prop['rtran'] = rtran
    
    for f1, f2 in zip(features, feat_avg):
        multi = Parallel(n_jobs=4)(delayed(knn_avg)(split_index[i], split_index[i+1], f1)
                                   for i in range(0,len(split_index)-1))
    
        single = [x for xx in multi for x in xx]
    
        ss2 = f2 #ss + "_" + f2
        prop[ss2] = single
        print ss, f2
        
    name = ss + ".csv"
    prop[["id_parcel", 'rmax', 'rmin', 'minmax_ratio', 'rtran'] + feat_avg].to_csv('../../'+name, index=False)

Folder k5


/home/ramezani/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:16: RuntimeWarning: divide by zero encountered in double_scalars


k5 area_lot_avg
k5 tax_property_avg
k5 build_year_avg
k5 area_total_calc_avg
k5 tax_building_avg
k5 tax_total_avg
